In [6]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d
from numpy.linalg import inv
import filterpy.kalman

In [7]:
import os
os.getcwd()
# This part is used to go to the directory of the video we are smoothing
os.chdir("C:\\Users\\18587\\Desktop\\Book Reading Analysis Project\\2D Data\\exports\\000")

In [8]:
import time
from numpy import ma
# We are smoothing the video by processing the gaze data. Name is the file that holds the gaze data in a csv file
name='gaze_positions'
data = pd.read_csv(name+'.csv')
xy = data[["norm_pos_x", "norm_pos_y"]].values
xy=ma.asarray(xy)
from pykalman import KalmanFilter
kf = KalmanFilter(initial_state_mean=[1,1.5],n_dim_obs=2)

start=0
stop=len(xy)
t1=time.time()
for i in range(1,stop):
    if ((xy[i,0]>1) or (xy[i,0]<0)or (xy[i,1]>1) or (xy[i,1]<0) or(abs(xy[i,0]-xy[i-1,0])>50/1280)or (abs(xy[i,1]-xy[i-1,1])>30/720)):
        xy[i,0]=ma.masked
        xy[i,1]=ma.masked
        a=0

t2=time.time()
print(t2-t1)
        
#output = kf.em(xy[start:stop]).smooth(xy[start:stop])
kf = kf.em(xy[start:stop], n_iter=5)
(filtered_state_means, filtered_state_covariances) = kf.filter(xy[start:stop])
(xy_new, smoothed_state_covariances) = kf.smooth(xy[start:stop])
t2=time.time()
print(t2-t1)
x_pred = xy_new[:,0]
y_pred = xy_new[:,1]


1.208463430404663
110.12842321395874


In [9]:
import csv

t=data["gaze_timestamp"]
w_ind=np.asarray(list(map(int,data['world_index'])))
print(type(w_ind))
conf=data["confidence"]
with open(name+'_filtered.csv', 'w') as f:
    f.write('timestamp,index,confidence,norm_pos_x,norm_pos_y\n')
    np.savetxt(f, np.column_stack((t, w_ind,conf,xy_new)), delimiter=",", fmt='%s')

<class 'numpy.ndarray'>


In [10]:
# %matplotlib notebook
# start=4000
# stop=6000
# fig = plt.figure()
# ax = plt.axes(projection='3d')
# ax.plot3D(data["gaze_timestamp"][start:stop],data["norm_pos_x"][start:stop], data["norm_pos_y"][start:stop])
# ax.set_zlim(0,1)
# ax.set_xlabel('timestamp')
# ax.set_ylabel('x')
# ax.set_zlabel('y');
# ax.set_title("Observed Values")
# ax.set_zlim(0,1)
# ax.set_ylim(0,1)
# plt.show()

# fig = plt.figure()
# ax = plt.axes(projection='3d')
# ax.plot3D(data["gaze_timestamp"][start:stop],x_pred[start:stop],y_pred[start:stop])
# ax.set_zlim(0,1)
# ax.set_ylim(0,1)

# ax.set_xlabel('timestamp')
# ax.set_ylabel('x')
# ax.set_zlabel('y');
# ax.set_title("After Kalman Smoothing")
# plt.show()